# Các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from time import sleep

In [2]:
# khai báo driver
driver = webdriver.Chrome()

In [3]:
# mở URL
driver.get("https://www.lazada.vn/dien-thoai-di-dong/?spm=a2o4n.home-vn.6598063730.37.19053bdcNwANax&up_id=2301126560&clickTrackInfo=491be505-9e8e-45ac-bbf5-f57141188de1__4518__2301126560__20__0.0989122__341738__7253__c2i__0.0__53%2525%2Boff&from=hp_categories&item_id=2301126560&version=v2&q=Mobiles&params=%7B%22catIdLv1%22%3A%224402%22%2C%22pvid%22%3A%22491be505-9e8e-45ac-bbf5-f57141188de1%22%2C%22src%22%3A%22ald%22%2C%22categoryName%22%3A%22Mobiles%22%2C%22categoryId%22%3A%224518%22%7D&src=hp_categories")
driver.implicitly_wait(10)

KeyboardInterrupt: 

In [4]:
elements_title = driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")
titles = [element_title.text for element_title in elements_title]
links = [element_title.get_attribute('href') for element_title in elements_title]

In [5]:
elements_discPrice = driver.find_elements(By.CSS_SELECTOR, ".aBrP0")
discPrices = [element_discPrice.text for element_discPrice in elements_discPrice]

In [6]:
elements_valDiscount = driver.find_elements(By.CSS_SELECTOR, ".IcOsH")
valDiscounts = [element_valDiscount.text for element_valDiscount in elements_valDiscount]

In [7]:
valDiscounts = [valDiscount.replace(' Off', '') for valDiscount in valDiscounts]

In [8]:
elements_sellAt = driver.find_elements(By.CSS_SELECTOR, "._6uN7R .oa6ri ")
sellAts = [element_sellAt.get_attribute('title') for element_sellAt in elements_sellAt]

In [9]:
df = pd.DataFrame(list(zip(titles, discPrices, valDiscounts, sellAts, links)), 
                  columns = ['title', 'discount_Price', 'value_Discount', 'sell_At', 'link_Item'])

df['index_'] = np.arange(1, len(df) + 1)
df = df[['index_', 'title', 'discount_Price', 'value_Discount', 'sell_At', 'link_Item']]

In [10]:
df.to_csv('data_crawl_selenium.csv', index = False)